In [32]:
from functools import reduce
import pandas as pd
import re, os

from extractor import getAppData

In [33]:
app_label = "com.example.cadprodutos"
app_id = "u0a380"

# data_dir = './measurements/makeFile'
# data_dir = './measurements/localLoginSharedPref'
data_dir = './measurements/file30'

In [34]:
df = []

def deep_get(dictionary, keys, default=None):
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)


for folder in os.scandir(data_dir):
    benchmark, iterations, filesize = folder.name.split('-')[:3]
    # benchmark, runs, file = folder.name.split('-')

    benchmark = f"{benchmark}-{iterations}-{filesize}"
    
    iterations = 100

    for i in range(1, iterations):
        data = getAppData(f"{folder.path}/energy-{i}.txt", app_id, log=False)

        if not data: continue

        df.append([
            # benchmark,
            # runs,
            benchmark,
            deep_get(data, 'time', default=0),
            deep_get(data, 'charge', default=0),
            deep_get(data, 'energy.total', default=0),
            deep_get(data, 'energy.cpu', default=0),
            deep_get(data, 'energy.wifi', default=0),
        ])

# df = pd.DataFrame(df, columns=['benchmark', 'runs', 'time', 'energy_total', 'energy_cpu', 'energy_wifi'])
df = pd.DataFrame(df, columns=['benchmark', 'time', 'charge', 'mAh_total', 'mAh_cpu', 'mAh_wifi'])

In [35]:
df

,benchmark,time,charge,mAh_total,mAh_cpu,mAh_wifi
0,makeEncFile-30-file30mb.txt,7692.0,2.540,1.270,1.270,0.00160
1,makeEncFile-30-file30mb.txt,3337.0,2.540,1.330,1.330,0.00111
2,makeEncFile-30-file30mb.txt,3475.0,2.540,1.330,1.330,0.00111
3,makeEncFile-30-file30mb.txt,3414.0,2.540,1.330,1.320,0.00111
4,makeEncFile-30-file30mb.txt,3433.0,2.540,1.400,1.400,0.00111
...,...,...,...,...,...,...
110,makeFile-30-file30mb.txt,3026.0,2.406,0.777,0.776,0.00111
111,makeFile-30-file30mb.txt,2928.0,2.406,0.697,0.695,0.00123
112,makeFile-30-file30mb.txt,3266.0,2.406,0.750,0.749,0.00111
113,makeFile-30-file30mb.txt,3406.0,2.406,0.748,0.747,0.00123


In [36]:
df['energy_total'] = df['mAh_total'] * 3.6 * df['charge']
df['energy_cpu'] = df['mAh_cpu'] * 3.6 * df['charge']
df['energy_wifi'] = df['mAh_wifi'] * 3.6 * df['charge']
df

,benchmark,time,charge,mAh_total,mAh_cpu,mAh_wifi,energy_total,energy_cpu,energy_wifi
0,makeEncFile-30-file30mb.txt,7692.0,2.540,1.270,1.270,0.00160,11.612880,11.612880,0.014630
1,makeEncFile-30-file30mb.txt,3337.0,2.540,1.330,1.330,0.00111,12.161520,12.161520,0.010150
2,makeEncFile-30-file30mb.txt,3475.0,2.540,1.330,1.330,0.00111,12.161520,12.161520,0.010150
3,makeEncFile-30-file30mb.txt,3414.0,2.540,1.330,1.320,0.00111,12.161520,12.070080,0.010150
4,makeEncFile-30-file30mb.txt,3433.0,2.540,1.400,1.400,0.00111,12.801600,12.801600,0.010150
...,...,...,...,...,...,...,...,...,...
110,makeFile-30-file30mb.txt,3026.0,2.406,0.777,0.776,0.00111,6.730063,6.721402,0.009614
111,makeFile-30-file30mb.txt,2928.0,2.406,0.697,0.695,0.00123,6.037135,6.019812,0.010654
112,makeFile-30-file30mb.txt,3266.0,2.406,0.750,0.749,0.00111,6.496200,6.487538,0.009614
113,makeFile-30-file30mb.txt,3406.0,2.406,0.748,0.747,0.00123,6.478877,6.470215,0.010654


In [37]:
df.to_csv('file30.csv', index=False, header=False, mode='a')